### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/" 
- 사외 테스트 환경 (LDAP 로그인 불가)
> URI = "https://web.aic-dev.lgebigdata.com/" 

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [ ]:
# 시각화 용 모듈 install 
!pip install tabulate ipywidgets

In [21]:
## 설정 파일로 별도 관리 예정
AIC_SETUP = {

    ## 인프라 setup
    # 'AIC_URI': "http://10.158.2.243:9999/", #"http://10.158.2.243:9999/", #"https://web.aic-dev.lgebigdata.com/", #"https://web.aic-dev.lgebigdata.com/", #"https://aic-kic.aidxlge.com/", #"http://10.158.2.243:9999/", 
    'AIC_URI': "https://web.aic-dev.lgebigdata.com/", #"http://10.158.2.243:9999/", #"https://web.aic-dev.lgebigdata.com/", #"https://web.aic-dev.lgebigdata.com/", #"https://aic-kic.aidxlge.com/", #"http://10.158.2.243:9999/", 
    "REGION": "ap-northeast-2",
    "WORKSPACE_NAME": "cism-ws",  ## magna-ws, cism-ws
    "INFERENCE_RESOURCE": "amd",  # amd, arm  

    ## 실행모드 setup 
    "LOGIN_MODE": "static", ## ldap, static
    "BUILD_METHOD": "docker",  ## docker, buildah 
    
    ## invisible
    "ECR_TAG": "latest",
    "SOLUTION_TYPE": "private", ##public, private 
    "BUILDAH_TAGS": [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ],
}
AIC_API_URI = {  ## !!! string 끝에 '/' 가 없어야 한다. (오동작 원인)
    # 0. 로그인
    'STATIC_LOGIN': 'api/v1/auth/static/login', # POST
    'LDAP_LOGIN': 'api/v1/auth/ldap/login',
    # 1. 시스템 정보 획득
    'SYSTEM_INFO':'api/v1/workspaces', # GET
    # 2. AI Solution 이름 설정 / 3. AI Solution 등록
    'AI_SOLUTION': 'api/v1/solutions/workspace', # 이름 설정 시 GET, 등록 시 POST
    # 4. AI Solution Instance 등록
    'SOLUTION_INSTANCE': 'api/v1/instances', # POST
    # 5. Stream 등록
    'STREAMS': 'api/v1/streams' # POST
}


#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요
    'login_id': "cism-dev", #"magna-dev", #'ws.jang', # "cism-dev"
    'login_pw': "cism-dev@com", #'magna-dev@com', # "cism-dev
    'solution_name': "test_12346",
    'user_parameters': "",
    'solution_type': "single", # "single", "multi"
    'selected_icon': "icon_id",  ## save icon_id
    'description': {},
    'icon_name': 'ic_bolt_fastening_inspection'
}

ds

TODO 컨텐츠 1회 실행하면 input folder 에서 데이터 가져옴을 명시하기 



In [33]:
import sys
from IPython.display import SVG, display
import os 

try:
    del sys.modules['register_utils_ssh'] 
except:
    pass
from register_utils_ssh import SolutionRegister

register = SolutionRegister(cloud_setup=AIC_SETUP, api_uri=AIC_API_URI)
register.debugging = False

## 로그인
register.login(user_input["login_id"], user_input["login_pw"])


#####################
### Cell 나누기 
#####################
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

if not user_input["solution_name"]:
    # soltuion 이름 입력 받기 
    display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
    name = input("- Enter the name you want to create: ")
else:
    name = user_input["solution_name"]

# check workspace (ECR, S3정보 셋팅까지 진행)
register.load_system_resource() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
register._init_solution_metadata()

#####################
### Cell 나누기 
#####################
register.check_solution_name(name)

#####################
### Cell 나누기 
#####################
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
html_content = register.s3_upload_icon_display()
# display(HTML(html_content))  ## 테스트중에는 꺼둠
register.s3_upload_icon(name=user_input["icon_name"])

#####################
### Cell 나누기 -- Set Train 
#####################

# s3 접근확인 & upload train data
register.s3_access_check('data')


register._sm_append_pipeline(pipeline_name='train')
register.s3_upload_data()
register.s3_upload_artifacts()

## train docker container 를 ecr 에 업로드
register.make_docker_container()

# register.set_resource(resource='standard')  ## resource 선택은 spec-out 됨

[SYSTEM] S3 key 파일을 로드 합니다. (file: /nas001/users/ruci.sung/aws.key)

#########################################
#######    Login to AI Conductor
#########################################

해당 계정으로 접근 가능한 workspace list: ['cism-ws']
[SYSTEM] 접근 요청하신 workspace (cism-ws) 은 해당 계정으로 접근 가능합니다.

#########################################
#######    Check ECR & S3 Resource
#########################################

[SYSTEM] AWS ECR:  
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/cism/
[SYSTEM] AWS S3 buckeet:  
s3-an2-hyunsoo-dev-cism

#########################################
#######    Solution Name Creation
#########################################

https://web.aic-dev.lgebigdata.com/api/v1/solutions/workspace
{'access-token': '5yXjj0eiyc83BRyQO8ojgDZlfj64hmX24vBcZ_PlQrk'}
<Response [200]>
[SYSTEM] 입력하신 Solution Name (test_12346)은 사용 가능합니다. (Success)
[SYSTEM] Solution Name List (in-use):
╒════╤══════════════════════════╕
│    │ AI solutions           

WARNING! Your password will be stored unencrypted in /home/sehyun.song/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[SYSTEM] AWS ECR | docker login result:
Login Succeeded

[SYSTEM] Target AWS ECR repository:
ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/test_12346/train/test_12346
[SYSTEM] AWS ECR create-repository response: 
{'repository': {'repositoryArn': 'arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/test_12346/train/test_12346', 'registryId': '086558720570', 'repositoryName': 'ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/test_12346/train/test_12346', 'repositoryUri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/cism/ai-solutions/test_12346/train/test_12346', 'createdAt': datetime.datetime(2024, 1, 16, 9, 18, 12, 390000, tzinfo=tzlocal()), 'imageTagMutability': 'MUTABLE', 'imageScanningConfiguration': {'scanOnPush': False}, 'encryptionConfiguration': {'encryptionType': 'AES256'}}, 'ResponseMetadata': {'RequestId': '40d9d995-3712-4107-853b-fc2f55dfae4d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '40d9d995-

TODO 진행할 사항 아래에 남겨두기 

In [5]:

# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'title': "UI solution title",
    
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)

# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler()

#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# Edge관련 정보 등록
edgeconductor_interface = {
            'support_labeling': False,
            
            'inference_result_datatype': 'table', # 'image'
            
            'train_datatype': 'table' # 'image'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)
#################################### 

## For Train 

## user parameters 입력
registerer.set_user_parameters(user_parameters)
## artifact 저장 경로 지정
registerer.set_artifacts_uri()

## For Inference 

###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()

registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


# DOCKERFILE setting
registerer.set_docker_contatiner()
## ECR 등록
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
docker_or_buildah = 'docker'
#----------------------------------------#

if docker_or_buildah == 'docker':
    ## docker login 실행 
    registerer.set_aws_ecr(docker=True)
    
elif docker_or_buildah == 'buildah':
    ## buildah login 실행 (docker in docker) 
    tags = [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ]
    registerer.set_aws_ecr(docker=False, tags=tags) 
# docker build 
registerer.build_docker()
registerer.docker_push()
registerer.set_container_uri() # uri도 그냥 입력되게 수정
## user parameters 입력
registerer.set_user_parameters(user_parameters)
## artifact 경로 설정 
registerer.set_artifacts_uri()



## only inference
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

inference_resource = 'standard'
registerer.set_resource(inference_resource)


## Registration 

registerer.register_solution()

NameError: name 'registerer' is not defined

In [ ]:
## Extra Job 

registerer.register_solution_instance()
registerer.register_stream()
registerer.request_run_stream()
registerer.get_stream_status()
registerer.download_artifacts()

# 가상환경이 잘 connected 돼있는지 확인 
!which python 
# 현재 작업경로 확인 
!pwd

# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference

----

#### **STEE-2-3**. 솔루션 사용자가 변경 가능한 User parameters를 등록합니다.  

# TODO 
- preprocess asset의 custom 같은 arg는 어찌 할 지 ? 
>- mode: custom \
>custom: {category_columns: [Pclass, SibSp], \
>            handle_missing: {fill_mean: [Age,Fare]}, \
>            numeric_scaler: {standard: [Fare], minmax: [Age]} \
>            }

In [ ]:
# TODO 아무 user parameter도 선택안했을 시에도 잘 작동하는지 테스트 필요 
# FIXME x_columns 같이 사용자가 몇개입력할지 모르는 건 string 으로 해야한다는걸 가이드 잘 할필요 있음 
from register_utils import RegisterUtils
import ipywidgets as widgets

candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', step_args['step'])
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)

In [ ]:
# FIXME boolean은 single selection이라 생각하고 유저 가이드도 필요할듯
# FIXME x_columns 같은 string과 multi-selection의 경계가 애매 ? > x_columns는 몇개가 될지 모르니 selection이 아니네 
# args type input 
type_dropdown_dict = {}

type_list = ['float', 'int', 'string', 'single_selection', 'multi_selection']
for step, checkboxes in args_checkboxes_dict.items(): # cbs: checkboxs
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    if len(selected_args) == 0: 
        continue 
    print('\n- step: ', step)
    for arg in selected_args:         
        dropdown = widgets.Dropdown(options=type_list, description=arg, value=None, style={'description_width': 'initial'})
        type_dropdown_dict[arg] = dropdown
        display(dropdown)

In [ ]:
# FIXME checkbox 중복 선택한 경우 처리하는 로직 필요할듯 ? 
# FIXME range는 포함관계 (이하,이상,미만,초과) 어떻게 되는지 ?
# FIXME range 같은 건 사용자가 , 로 분리해서 써야된다고 잘 가이드 필요 
# FIXME x_columns 처럼 default None 인거 어떻게 처리? (일단 None으로 했음)
def get_arg_format(arg_type: str):
    if arg_type in ['int', 'float', 'string']:
        return ['name', 'description', 'type', 'default', 'range']
    elif arg_type in ['single_selection', 'multi_selection']: 
        return ['name', 'description', 'type', 'selectable', 'default']
    else: 
        raise ValueError(f"Unsupported type of user paramter: << {arg_type} >>") 
    
def get_text_widgets(arg_format, arg_name, arg_type):
    text_widgets = [] 
    for desc in arg_format: 
        if desc == 'name': 
            text_widgets.append(widgets.Text(description=desc, value=arg_name))
        elif desc == 'type': 
            text_widgets.append(widgets.Text(description=desc, value=arg_type))
        else: 
            text_widgets.append(widgets.Text(description=desc))
    return text_widgets

accordion_list = []
for step, checkboxes in args_checkboxes_dict.items(): 
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    text_widgets_list = [] 
    for arg in selected_args: 
        selected_type = type_dropdown_dict[arg].value
        assert selected_type is not None 
        arg_format = get_arg_format(selected_type)
        text_widgets_list.append(get_text_widgets(arg_format=arg_format, arg_name=arg, arg_type=selected_type))
        
    accordion = widgets.Accordion(children=[widgets.VBox(children=text_widgets) for text_widgets in text_widgets_list])
    for idx, arg in enumerate(selected_args):
        accordion.set_title(idx, arg)
    accordion_list.append((step, accordion)) # [[step, accordion]]
    
tab_nest = widgets.Tab()
tab_nest.children = [step_accordion[1] for step_accordion in accordion_list]
for idx, step_accordion in enumerate(accordion_list): 
    tab_nest.set_title(idx, step_accordion[0])
display(tab_nest)

In [ ]:
from register_utils import convert_args_type 
# solution_metadata.yaml에 user parameters 셋팅 
user_parameters = [] 
for step, accordion in accordion_list: 
    args_list = []
    if len(accordion.children) != 0:
        for vbox in accordion.children: # vbox 하나가 arg 하나에 대응됨  
            args = {tbox.description: tbox.value for tbox in vbox.children}
            args_list.append(convert_args_type(args))
    user_parameters.append({'step': step, 'args': args_list})


----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [ ]:
candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', step_args['step'])
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)

In [ ]:
# FIXME boolean은 single selection이라 생각하고 유저 가이드도 필요할듯
# FIXME x_columns 같은 string과 multi-selection의 경계가 애매 ? > x_columns는 몇개가 될지 모르니 selection이 아니네 
# args type input 
type_dropdown_dict = {}

type_list = ['float', 'int', 'string', 'single_selection', 'multi_selection']
for step, checkboxes in args_checkboxes_dict.items(): # cbs: checkboxs
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    if len(selected_args) == 0: 
        continue 
    print('\n- step: ', step)
    for arg in selected_args:         
        dropdown = widgets.Dropdown(options=type_list, description=arg, value=None, style={'description_width': 'initial'})
        type_dropdown_dict[arg] = dropdown
        display(dropdown)

In [ ]:
accordion_list = []
for step, checkboxes in args_checkboxes_dict.items(): 
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    text_widgets_list = [] 
    for arg in selected_args: 
        selected_type = type_dropdown_dict[arg].value
        assert selected_type is not None 
        arg_format = get_arg_format(selected_type)
        text_widgets_list.append(get_text_widgets(arg_format=arg_format, arg_name=arg, arg_type=selected_type))
        
    accordion = widgets.Accordion(children=[widgets.VBox(children=text_widgets) for text_widgets in text_widgets_list])
    for idx, arg in enumerate(selected_args):
        accordion.set_title(idx, arg)
    accordion_list.append((step, accordion)) # [[step, accordion]]
    
tab_nest = widgets.Tab()
tab_nest.children = [step_accordion[1] for step_accordion in accordion_list]
for idx, step_accordion in enumerate(accordion_list): 
    tab_nest.set_title(idx, step_accordion[0])
display(tab_nest)

In [ ]:
# solution_metadata.yaml에 user parameters 셋팅 
user_parameters = [] 
for step, accordion in accordion_list: 
    args_list = []
    if len(accordion.children) != 0:
        for vbox in accordion.children: # vbox 하나가 arg 하나에 대응됨  
            args = {tbox.description: tbox.value for tbox in vbox.children}
            args_list.append(convert_args_type(args))
    user_parameters.append({'step': step, 'args': args_list})